In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp experiments

In [ ]:
#| export
import os, json, csv
from comet_ml import API, APIExperiment, start
from pct.hierarchy import PCTHierarchy  
from comet_ml.query import Metric

In [ ]:
#| export
class CometExperimentManager:
    def __init__(self, api_key: str = None, workspace: str = None):
        self.api = API(api_key)
        self.workspace = workspace

    def get_all_artifacts_indexed(self):
        """Retrieve all artifacts and sort them by source experiment key."""
        filename = '/tmp/artifacts/artifacts_results.json'
        if os.path.exists(filename):
            with open(filename, 'r') as file:
                return json.load(file)
            
        artifacts = self.api.get_artifact_list(workspace=self.workspace)
        experiment = start(workspace=self.workspace)
        results = {}
        for artifact_dict in artifacts['artifacts']:
            id = artifact_dict['name']
            try:
                logged_artifact = experiment.get_artifact(id)
                print(logged_artifact.source_experiment_key, id)
                results[logged_artifact.source_experiment_key] = id
            except Exception as e:
                print(f"Error retrieving artifact {id}: {e}")

        # Save results to a file
        with open(filename, "w") as file:
            json.dump(results, file, indent=4)
        experiment.end()
        return results

    def get_experiments_by_metrics(self, project_name: str = None, score_threshold: float = None, reward_threshold: float = None):
        """
        Retrieve experiments for a project where the metric 'score' is less than
        score_threshold and 'reward' is greater than or equal to reward_threshold.
        """
        # experiments = self.api.query(self.workspace, project_name, Metric("score") < score_threshold & Metric("reward") >= reward_threshold)
        experiments = self.api.query(self.workspace, project_name, Metric("score") < score_threshold )
 
        return experiments

    def get_artifact_name(self, experiment: APIExperiment = None):
        """Retrieve the name of an artifact from an experiment."""
        artifacts = experiment.get_artifacts()
        return artifacts[0]['artifact_name'] if artifacts else None

    def download_and_run_artifact(self, artifact_name: str = None, seeds: list[int] = None):
        """
        Download an artifact to '/tmp/artifacts/' and run PCTHierarchy.run_from_file
        with the artifact filename, returning the score value for each run.
        """
        download_path = f"/tmp/artifacts/"
        os.makedirs(os.path.dirname(download_path), exist_ok=True)
 
        full_path = os.path.join(download_path, artifact_name)
        if os.path.exists(full_path):
            pass
        else:
            experiment = start(workspace=self.workspace)
            logged_artifact  = experiment.get_artifact(artifact_name)
            # print(logged_artifact.source_experiment_key)
            # local_artifact = logged_artifact.download(download_path)
            logged_artifact.download(download_path)
            # filename = f"{local_artifact.download_local_path}{artifact_name}"
            experiment.end()
    
        rewards = []
        for seed in seeds:
            hierarchy, score = PCTHierarchy.run_from_file(full_path, seed=seed)
            metrics = hierarchy.get_environment().get_metrics()
            # print(f'Score={score:0.3f} {metrics}')
            rewards.append(metrics['reward'])



        return rewards

    def get_original_metrics(self, experiment: APIExperiment = None):
        """Retrieve the metrics for an experiment."""
        metrics = {}
        metrics['score'] = eval(experiment.get_metrics("score")[0]['metricValue'])
        hyperparameters = experiment.get_parameters_summary()

        for param in hyperparameters:
            if param['name'] == 'mode':
                metrics['mode'] = param['valueCurrent']
                break

        metrics['name'] = experiment.name
        return metrics

    def run_experiments_and_record_results(self, project_name: str = None, experiments: list[APIExperiment] = None, artifact_results: dict = None, num_runs: int = 0, output_csv: str = None):
        """
        Run each experiment for a given project a specified number of times and record the score and the
        number of times the reward is 100, -100, or something else. Save results to a CSV file.
        """
        results = []
        for experiment in experiments:
            artifact_name = artifact_results[experiment.id]

            metrics = self.get_original_metrics(experiment)

            print(f"Running experiment {experiment.id} in project {project_name} with artifact {artifact_name}")
            if not artifact_name:
                continue

            rewards = self.download_and_run_artifact(artifact_name, seeds=range(num_runs))
            reward_counts = {'100': 0, '-100': 0, 'other': 0}

            for reward in rewards:
                if reward == 100:
                    reward_counts['100'] += 1
                elif reward == -100:
                    reward_counts['-100'] += 1
                else:
                    reward_counts['other'] += 1
            print(f"Rewards: {reward_counts}")
            results.append({
                'name': metrics['name'],
                'score': round(metrics['score'], 5),
                'mode': metrics['mode'],
                'reward_100': reward_counts['100'],
                'reward_-100': reward_counts['-100'],
                'reward_other': reward_counts['other'],
                'experiment_key': '/'.join(("https://www.comet.com", self.workspace, project_name, experiment.id)),
                'artifact_name': artifact_name
            })
        # Sort results by 'reward_100' in descending order, then by 'reward_other' in descending order
        results.sort(key=lambda x: (-x['reward_100'], -x['reward_other']))
        # Save results to CSV
        with open(output_csv, mode='w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=['name', 'score', 'mode', 'reward_100', 'reward_-100', 'reward_other', 'experiment_key', 'artifact_name'])
            writer.writeheader()
            writer.writerows(results)

In [ ]:
#|gui
# Initialize the manager
workspace = 'lunarlandercontinuous-v2'
project_name = 'refinputs-smooth'
manager = CometExperimentManager(workspace=workspace)

# Test get_all_artifacts_sorted
artifact_results = manager.get_all_artifacts_indexed()
# print("Artifacts sorted by source experiment key:", artifacts)

# Test get_experiments_by_metrics
experiments = manager.get_experiments_by_metrics(project_name=project_name, score_threshold=0.05, reward_threshold=10.0)
print("Filtered experiments:", experiments)



Filtered experiments: [<APIExperiment 'lunarlandercontinuous-v2/refinputs-smooth/319b8777136d4e059261d6e1582cded9'>, <APIExperiment 'lunarlandercontinuous-v2/refinputs-smooth/e6751db7179d4d6483ffd82985285f3e'>, <APIExperiment 'lunarlandercontinuous-v2/refinputs-smooth/39dfeda830bd4cf59759368a36bccb67'>, <APIExperiment 'lunarlandercontinuous-v2/refinputs-smooth/70d024308d024f24bcf4d9a7352ca775'>, <APIExperiment 'lunarlandercontinuous-v2/refinputs-smooth/bd30363071b74bc98582b51107cd38de'>, <APIExperiment 'lunarlandercontinuous-v2/refinputs-smooth/c888399ea415447d9c9954ca1f83f73d'>, <APIExperiment 'lunarlandercontinuous-v2/refinputs-smooth/620552456f29486fa2140e7afeeea053'>, <APIExperiment 'lunarlandercontinuous-v2/refinputs-smooth/5f14659ba5654ca4a7e894207f1b14e3'>, <APIExperiment 'lunarlandercontinuous-v2/refinputs-smooth/1d922993d9f7437693c6d66e2a4aa78b'>, <APIExperiment 'lunarlandercontinuous-v2/refinputs-smooth/873ef105d9004b9f81cff6caed888a5a'>, <APIExperiment 'lunarlandercontinuous

In [ ]:
#|gui
# import random

# Test run_experiments_and_record_results
if experiments:
    output_csv = "/tmp/artifacts/experiment_results.csv"
    manager.run_experiments_and_record_results(experiments=experiments, project_name=project_name, artifact_results=artifact_results, num_runs=2, output_csv=output_csv)
    print(f"Results saved to {output_csv}")


Running experiment 319b8777136d4e059261d6e1582cded9 in project refinputs-smooth with artifact ga-000.009-s001-1x6-m000-LL0061-1d7c4567db2419cd0699fc7dacd1e567.properties
Rewards: {'100': 1, '-100': 1, 'other': 0}
Running experiment e6751db7179d4d6483ffd82985285f3e in project refinputs-smooth with artifact ga-000.011-s002-1x6-m001-LL0062-0a9e1c620271e5d3e0a02bcab106a413-consolidated.properties
Rewards: {'100': 1, '-100': 0, 'other': 1}
Running experiment 39dfeda830bd4cf59759368a36bccb67 in project refinputs-smooth with artifact ga-000.008-s003-1x6-m001-LL0062-d72a0f7fcd4502d784e75d34d3b5b8de-consolidated.properties
Rewards: {'100': 0, '-100': 2, 'other': 0}
Running experiment 70d024308d024f24bcf4d9a7352ca775 in project refinputs-smooth with artifact ga-000.013-s003-1x6-m002-LL0063-8b5293fa0a14cc94337149e8dcda75c9.properties
Rewards: {'100': 0, '-100': 2, 'other': 0}
Running experiment bd30363071b74bc98582b51107cd38de in project refinputs-smooth with artifact ga-000.016-s002-1x6-m002-LL0

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()